In [ ]:
# this notebook shows how I use backtrader to backtest the data from btc_processed.csv
# the codes are modified from https://www.backtrader.com/docu/quickstart/quickstart/
# I think you need to read this documentation to understand the strategy used

In [114]:
# The following code should be run in python 3.4, other version may be compatible
# !pip3 install Matplotlib
# !pip install backtrader
# !pip install backtrader[plotting]

In [115]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt
import os.path
import datetime

In [116]:
INITIAL_CASH = 1000000.0
DATAPATH = '../datas/btc_processed.csv'
FROM_DATE = datetime.datetime(2014, 1, 1)
TO_DATE = datetime.datetime(2020, 12, 31)

In [117]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [118]:
if __name__ == '__main__':
    data = bt.feeds.GenericCSVData(
        dataname = DATAPATH,
        datetime = 0,
        time = -1,
        open = 1,
        high = 2,
        low = 3,
        close = 4,
        volume = 5,
        openinterest = -1,
        nullvalue = float('NaN'),
        dtformat = ('%Y-%m-%d'),
        tmformat = ('%H:%M:%S')
    )
    
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(data)
    cerebro.broker.setcash(INITIAL_CASH)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000000.00
2014-01-15, Close, 860.90
2014-01-15, BUY CREATE, 860.90
2014-01-16, BUY EXECUTED, Price: 860.29, Cost: 8602.90, Comm 8.60
2014-01-16, Close, 835.63
2014-01-16, SELL CREATE, 835.63
2014-01-17, SELL EXECUTED, Price: 834.49, Cost: 8602.90, Comm 8.34
2014-01-17, OPERATION PROFIT, GROSS -258.00, NET -274.95
2014-01-17, Close, 814.64
2014-01-18, Close, 840.00
2014-01-19, Close, 870.96
2014-01-19, BUY CREATE, 870.96
2014-01-20, BUY EXECUTED, Price: 871.39, Cost: 8713.90, Comm 8.71
2014-01-20, Close, 870.20
2014-01-21, Close, 863.91
2014-01-22, Close, 845.59
2014-01-22, SELL CREATE, 845.59
2014-01-23, SELL EXECUTED, Price: 845.46, Cost: 8713.90, Comm 8.45
2014-01-23, OPERATION PROFIT, GROSS -259.30, NET -276.47
2014-01-23, Close, 822.04
2014-01-24, Close, 797.07
2014-01-25, Close, 853.61
2014-01-25, BUY CREATE, 853.61
2014-01-26, BUY EXECUTED, Price: 853.68, Cost: 8536.80, Comm 8.54
2014-01-26, Close, 885.28
2014-01-27, Close, 771.39
2014-01-27, SELL CREAT

2015-05-14, SELL EXECUTED, Price: 236.21, Cost: 2372.04, Comm 2.36
2015-05-14, OPERATION PROFIT, GROSS -9.90, NET -14.63
2015-05-14, Close, 236.93
2015-05-15, Close, 237.60
2015-05-16, Close, 236.15
2015-05-17, Close, 236.80
2015-05-18, Close, 233.13
2015-05-19, Close, 231.95
2015-05-20, Close, 234.02
2015-05-21, Close, 235.34
2015-05-22, Close, 240.35
2015-05-22, BUY CREATE, 240.35
2015-05-23, BUY EXECUTED, Price: 240.29, Cost: 2402.86, Comm 2.40
2015-05-23, Close, 238.87
2015-05-24, Close, 240.95
2015-05-25, Close, 237.11
2015-05-25, SELL CREATE, 237.11
2015-05-26, SELL EXECUTED, Price: 237.10, Cost: 2402.86, Comm 2.37
2015-05-26, OPERATION PROFIT, GROSS -31.82, NET -36.59
2015-05-26, Close, 237.12
2015-05-26, BUY CREATE, 237.12
2015-05-27, BUY EXECUTED, Price: 237.07, Cost: 2370.65, Comm 2.37
2015-05-27, Close, 237.28
2015-05-28, Close, 237.41
2015-05-29, Close, 237.10
2015-05-30, Close, 233.35
2015-05-30, SELL CREATE, 233.35
2015-05-31, SELL EXECUTED, Price: 233.13, Cost: 2370.65, 

2016-09-04, Close, 608.63
2016-09-05, Close, 606.59
2016-09-06, Close, 610.44
2016-09-07, Close, 614.54
2016-09-08, Close, 626.32
2016-09-09, Close, 622.86
2016-09-10, Close, 623.51
2016-09-11, Close, 606.72
2016-09-12, Close, 608.24
2016-09-13, Close, 609.24
2016-09-14, Close, 610.68
2016-09-15, Close, 607.16
2016-09-16, Close, 606.97
2016-09-16, SELL CREATE, 606.97
2016-09-17, SELL EXECUTED, Price: 607.22, Cost: 5985.90, Comm 6.07
2016-09-17, OPERATION PROFIT, GROSS 86.28, NET 74.22
2016-09-17, Close, 605.98
2016-09-18, Close, 609.87
2016-09-19, Close, 609.23
2016-09-20, Close, 608.31
2016-09-21, Close, 597.15
2016-09-22, Close, 596.30
2016-09-23, Close, 602.84
2016-09-24, Close, 602.62
2016-09-25, Close, 600.83
2016-09-26, Close, 608.04
2016-09-26, BUY CREATE, 608.04
2016-09-27, BUY EXECUTED, Price: 608.02, Cost: 6080.22, Comm 6.08
2016-09-27, Close, 606.17
2016-09-28, Close, 604.73
2016-09-28, SELL CREATE, 604.73
2016-09-29, SELL EXECUTED, Price: 605.02, Cost: 6080.22, Comm 6.05
20

2017-04-22, Close, 1231.71
2017-04-23, Close, 1207.21
2017-04-24, Close, 1250.15
2017-04-25, Close, 1265.49
2017-04-26, Close, 1281.08
2017-04-27, Close, 1317.73
2017-04-28, Close, 1316.48
2017-04-29, Close, 1321.79
2017-04-30, Close, 1347.89
2017-05-01, Close, 1421.60
2017-05-02, Close, 1452.82
2017-05-03, Close, 1490.09
2017-05-04, Close, 1537.67
2017-05-05, Close, 1555.45
2017-05-06, Close, 1578.80
2017-05-07, Close, 1596.71
2017-05-08, Close, 1723.35
2017-05-09, Close, 1755.36
2017-05-10, Close, 1787.13
2017-05-11, Close, 1848.57
2017-05-12, Close, 1724.24
2017-05-13, Close, 1804.91
2017-05-14, Close, 1808.91
2017-05-15, Close, 1738.43
2017-05-16, Close, 1734.45
2017-05-17, Close, 1839.09
2017-05-18, Close, 1888.65
2017-05-19, Close, 1987.71
2017-05-20, Close, 2084.73
2017-05-21, Close, 2041.20
2017-05-22, Close, 2173.40
2017-05-23, Close, 2320.42
2017-05-24, Close, 2443.64
2017-05-25, Close, 2304.98
2017-05-26, Close, 2202.42
2017-05-27, Close, 2038.87
2017-05-28, Close, 2155.80
2

2018-04-19, Close, 8294.31
2018-04-20, Close, 8845.83
2018-04-21, Close, 8895.58
2018-04-22, Close, 8802.46
2018-04-23, Close, 8930.88
2018-04-24, Close, 9697.50
2018-04-25, Close, 8845.74
2018-04-26, Close, 9281.51
2018-04-27, Close, 8987.05
2018-04-28, Close, 9348.48
2018-04-29, Close, 9419.08
2018-04-30, Close, 9240.55
2018-05-01, Close, 9119.01
2018-05-02, Close, 9235.92
2018-05-03, Close, 9743.86
2018-05-04, Close, 9700.76
2018-05-05, Close, 9858.15
2018-05-06, Close, 9654.80
2018-05-07, Close, 9373.01
2018-05-08, Close, 9234.82
2018-05-08, SELL CREATE, 9234.82
2018-05-09, SELL EXECUTED, Price: 9223.73, Cost: 79010.90, Comm 92.24
2018-05-09, OPERATION PROFIT, GROSS 13226.41, NET 13055.16
2018-05-09, Close, 9325.18
2018-05-10, Close, 9043.94
2018-05-11, Close, 8441.49
2018-05-12, Close, 8504.89
2018-05-13, Close, 8723.94
2018-05-14, Close, 8716.79
2018-05-15, Close, 8510.38
2018-05-16, Close, 8368.83
2018-05-17, Close, 8094.32
2018-05-18, Close, 8250.97
2018-05-19, Close, 8247.18
2

2020-02-22, Close, 9663.18
2020-02-23, Close, 9924.52
2020-02-24, Close, 9650.17
2020-02-25, Close, 9341.71
2020-02-26, Close, 8820.52
2020-02-27, Close, 8784.49
2020-02-28, Close, 8672.46
2020-02-29, Close, 8599.51
2020-03-01, Close, 8562.45
2020-03-02, Close, 8869.67
2020-03-03, Close, 8787.79
2020-03-04, Close, 8755.25
2020-03-05, Close, 9078.76
2020-03-06, Close, 9122.55
2020-03-06, BUY CREATE, 9122.55
2020-03-07, BUY EXECUTED, Price: 9121.60, Cost: 91216.01, Comm 91.22
2020-03-07, Close, 8909.95
2020-03-07, SELL CREATE, 8909.95
2020-03-08, SELL EXECUTED, Price: 8908.21, Cost: 91216.01, Comm 89.08
2020-03-08, OPERATION PROFIT, GROSS -2133.95, NET -2314.24
2020-03-08, Close, 8108.12
2020-03-09, Close, 7923.64
2020-03-10, Close, 7909.73
2020-03-11, Close, 7911.43
2020-03-12, Close, 4970.79
2020-03-13, Close, 5563.71
2020-03-14, Close, 5200.37
2020-03-15, Close, 5392.31
2020-03-16, Close, 5014.48
2020-03-17, Close, 5225.63
2020-03-18, Close, 5238.44
2020-03-19, Close, 6191.19
2020-03-